In [1]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

Warning message:
"package 'dplyr' was built under R version 3.6.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'caret' was built under R version 3.6.3"
Loading required package: lattice

Warning message:
"package 'lattice' was built under R version 3.6.2"
Loading required package: ggplot2

Warning message:
"package 'tidyverse' was built under R version 3.6.3"
-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

v tibble  3.1.0     v purrr   0.3.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

Warning message:
"package 'tibble' was built under R version 3.6.2"
Warning message:
"package 'tidyr' was built under R version 3.6.3"
Warning message:
"package 'readr' was built under R version 3.6.3"
Warning message:
"pack

In [2]:
dim(df_finale)

[1] 1148   77

In [3]:
tmp <- subset(df, select = -c(deg_cat))           #hot encoding

#define one-hot encoding function
dummy <- dummyVars(" ~ .", data=tmp)

#perform one-hot encoding on data frame
final_df <- data.frame(predict(dummy, newdata=tmp))


final_df$deg_cat <- df$deg_cat


df <- final_df

## Data partition

In [4]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)



TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [5]:
dim(test)
dim(train)

[1] 344 181

[1] 804 181

In [6]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    401     402 

## Model

In [7]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [8]:

model_svmPoly <- train(x=TrainData, y=TrainClasses, 
               method = "svmPoly", scale= FALSE) 

In [9]:

model_svmRadial <- train(x=TrainData, y=TrainClasses, 
               method = "svmRadial", scale= FALSE) 

Warning message in .local(x, ...):
"Variable(s) `' constant. Cannot scale data."


In [10]:

model_svmLinear <- train(x=TrainData, y=TrainClasses, 
               method = "svmLinear", scale= FALSE)

# Results

In [11]:
print(model_svmPoly)
plot(model_svmPoly,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Support Vector Machines with Polynomial Kernel 

803 samples
180 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 803, 803, 803, 803, 803, 803, ... 
Resampling results across tuning parameters:

  degree  scale  C     Accuracy   Kappa    
  1       0.001  0.25  0.5826788  0.1820337
  1       0.001  0.50  0.7141073  0.4321281
  1       0.001  1.00  0.7335230  0.4690330
  1       0.010  0.25  0.7426268  0.4866447
  1       0.010  0.50  0.7439877  0.4889676
  1       0.010  1.00  0.7423408  0.4854125
  1       0.100  0.25  0.7348267  0.4704431
  1       0.100  0.50  0.7280173  0.4567325
  1       0.100  1.00  0.7232604  0.4471878
  2       0.001  0.25  0.7150758  0.4339110
  2       0.001  0.50  0.7342005  0.4703906
  2       0.001  1.00  0.7416934  0.4848456
  2       0.010  0.25  0.7452286  0.4912230
  2       0.010  0.50  0.7437594  0.4883011
  2       0.010  1.00  0.7362096  0.4732418
  2       0.100  0.25  0.75

In [12]:
print(model_svmRadial)
plot(model_svmRadial,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Support Vector Machines with Radial Basis Function Kernel 

803 samples
180 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 803, 803, 803, 803, 803, 803, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.7530254  0.5054429
  0.50  0.7576420  0.5143743
  1.00  0.7630074  0.5253100

Tuning parameter 'sigma' was held constant at a value of 0.01784576
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.01784576 and C = 1.


In [13]:
print(model_svmLinear)
plot(model_svmLinear)

Support Vector Machines with Linear Kernel 

803 samples
180 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 803, 803, 803, 803, 803, 803, ... 
Resampling results:

  Accuracy   Kappa    
  0.7157011  0.4309171

Tuning parameter 'C' was held constant at a value of 1


ERROR: Error in plot.train(model_svmLinear): There are no tuning parameters with more than 1 value.


In [14]:
# collect resamples
results <- resamples(list(Poly=model_svmPoly,  Radial=model_svmRadial, Linear=model_svmLinear))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: Poly, Radial, Linear 
Number of resamples: 25 

Accuracy 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
Poly   0.7024221 0.7467105 0.7572816 0.7577810 0.7702265 0.7928803    0
Radial 0.7344262 0.7508532 0.7607973 0.7630074 0.7746479 0.8068966    0
Linear 0.6712329 0.7027027 0.7147541 0.7157011 0.7333333 0.7593220    0

Kappa 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
Poly   0.4123794 0.4914842 0.5143851 0.5162004 0.5410923 0.5850434    0
Radial 0.4667717 0.5016659 0.5202975 0.5253100 0.5501658 0.6084294    0
Linear 0.3394910 0.4014706 0.4251641 0.4309171 0.4670220 0.5195550    0


# Prediction

In [15]:
pred_svmPoly <- predict(model_svmPoly,TestData)
pred_svmPoly.prob <- predict(model_svmPoly,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmPoly) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      73      48
   NotFast   56     167
                                          
               Accuracy : 0.6977          
                 95% CI : (0.6461, 0.7458)
    No Information Rate : 0.625           
    P-Value [Acc > NIR] : 0.002853        
                                          
                  Kappa : 0.3469          
                                          
 Mcnemar's Test P-Value : 0.492457        
                                          
            Sensitivity : 0.5659          
            Specificity : 0.7767          
         Pos Pred Value : 0.6033          
         Neg Pred Value : 0.7489          
             Prevalence : 0.3750          
         Detection Rate : 0.2122          
   Detection Prevalence : 0.3517          
      Balanced Accuracy : 0.6713          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5658915            0.7767442            0.6033058 
      Neg Pred Value            Precision               Recall 
           0.7488789            0.6033058            0.5658915 
                  F1           Prevalence       Detection Rate 
           0.5840000            0.3750000            0.2122093 
Detection Prevalence    Balanced Accuracy 
           0.3517442            0.6713178

In [16]:
pred_svmRadial <- predict(model_svmRadial,TestData)
pred_svmRadial.prob <- predict(model_svmRadial,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmRadial) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      84      37
   NotFast   60     163
                                         
               Accuracy : 0.718          
                 95% CI : (0.6673, 0.765)
    No Information Rate : 0.5814         
    P-Value [Acc > NIR] : 1.032e-07      
                                         
                  Kappa : 0.4074         
                                         
 Mcnemar's Test P-Value : 0.0255         
                                         
            Sensitivity : 0.5833         
            Specificity : 0.8150         
         Pos Pred Value : 0.6942         
         Neg Pred Value : 0.7309         
             Prevalence : 0.4186         
         Detection Rate : 0.2442         
   Detection Prevalence : 0.3517         
      Balanced Accuracy : 0.6992         
                                         
       'Positive' Class : fast           
                                   

Sensitivity          Specificity       Pos Pred Value 
           0.5833333            0.8150000            0.6942149 
      Neg Pred Value            Precision               Recall 
           0.7309417            0.6942149            0.5833333 
                  F1           Prevalence       Detection Rate 
           0.6339623            0.4186047            0.2441860 
Detection Prevalence    Balanced Accuracy 
           0.3517442            0.6991667

In [17]:
pred_svmLinear <- predict(model_svmLinear,TestData)
pred_svmLinear.prob <- predict(model_svmLinear,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmLinear) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      81      40
   NotFast   58     165
                                          
               Accuracy : 0.7151          
                 95% CI : (0.6643, 0.7622)
    No Information Rate : 0.5959          
    P-Value [Acc > NIR] : 2.789e-06       
                                          
                  Kappa : 0.3959          
                                          
 Mcnemar's Test P-Value : 0.08593         
                                          
            Sensitivity : 0.5827          
            Specificity : 0.8049          
         Pos Pred Value : 0.6694          
         Neg Pred Value : 0.7399          
             Prevalence : 0.4041          
         Detection Rate : 0.2355          
   Detection Prevalence : 0.3517          
      Balanced Accuracy : 0.6938          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5827338            0.8048780            0.6694215 
      Neg Pred Value            Precision               Recall 
           0.7399103            0.6694215            0.5827338 
                  F1           Prevalence       Detection Rate 
           0.6230769            0.4040698            0.2354651 
Detection Prevalence    Balanced Accuracy 
           0.3517442            0.6938059

In [18]:
roc_obj <- roc(TestClasses, as.numeric(pred_svmRadial))
plot(roc_obj, main="Curva ROC")


Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [19]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_svmRadial))

Data: as.numeric(pred_svmRadial) in 121 controls (TestClasses fast) < 223 cases (TestClasses NotFast).
Area under the curve: 0.7126

In [20]:
pred_svmRadial.prob

fast,NotFast
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
